In [20]:
# SELECT *
# FROM 
#     hz_parties,
#     hz_party_sites
# WHERE 
#     hz_parties.party_id = hz_party_sites.party_id;
from pyspark.sql import SparkSession
import pyarrow.parquet as pq
import pandas as pd

SyntaxError: invalid syntax (<ipython-input-11-1459640cebba>, line 1)

In [16]:
# !pip install --upgrade setuptools wheel

!pip install pyspark

  Using cached pyspark-3.5.0.tar.gz (316.9 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425384 sha256=03c9976e79260930c4e6ac6be6f1d414ccc309c7b904054dbbae9107bddfd582

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063



  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\a6\ce\f9\17d82c92f044018df2fe30af63ac043447720d5b2cee39b40f
Successfully built pyspark


In [18]:
%%time

# Read the Parquet files into Arrow tables
hz_parties = pq.read_table('hz_parties.parquet') #361919
hz_party_sites = pq.read_table('hz_party_sites.parquet') #321247

# # Convert the Arrow tables to Pandas DataFrames
df1 = hz_parties.to_pandas()
df2 = hz_party_sites.to_pandas()

# # Merge the two DataFrames, if needed

merged_df = pd.merge(df1, df2, on="PARTY_ID")
print(merged_df) #321245   

        PARTY_ID PARTY_NUMBER  \
0          30871        15146   
1          30872        15147   
2          30873        15148   
3          30874        15149   
4          30874        15149   
...          ...          ...   
321240   2912188       232999   
321241   2912189       233000   
321242   2912190       233001   
321243   2912191       233002   
321244   2912192       233003   

                                             PARTY_NAME    PARTY_TYPE  \
0                                                  CDFA  ORGANIZATION   
1                        FULTON COUNTY TAX COMMISSIONER  ORGANIZATION   
2                                               WHY INC  ORGANIZATION   
3                                  M&G PARTNERS LLP DBA  ORGANIZATION   
4                                  M&G PARTNERS LLP DBA  ORGANIZATION   
...                                                 ...           ...   
321240                       GRANDADDY'S BBQ - DAVIDSON  ORGANIZATION   
321241             

In [21]:
%%time
spark = SparkSession.builder.appName("ParquetJoinExample").getOrCreate()

df1 = spark.read.parquet("hz_parties.parquet")
df2 = spark.read.parquet("hz_party_sites.parquet")

result_df = df1.join(df2, "PARTY_ID")


Py4JJavaError: An error occurred while calling o25.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (DESKTOP-63H1IAF executor driver): org.apache.spark.sql.AnalysisException: Illegal Parquet type: INT64 (TIMESTAMP(NANOS,false)).
	at org.apache.spark.sql.errors.QueryCompilationErrors$.illegalParquetTypeError(QueryCompilationErrors.scala:1826)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.illegalType$1(ParquetSchemaConverter.scala:206)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.$anonfun$convertPrimitiveField$2(ParquetSchemaConverter.scala:283)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.convertPrimitiveField(ParquetSchemaConverter.scala:224)
	at org.apache.spark.sql.execution.datasources.parquet.Par

In [ ]:
from pyspark.sql.types import StructType, StructField, TimestampType

custom_schema = StructType([
    StructField("your_timestamp_column", TimestampType(), True),
    # Add other fields as needed
])

df = spark.read.schema(custom_schema).parquet("your_parquet_file.parquet")


In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, TimestampType

# Create a Spark session
spark = SparkSession.builder.appName("ParquetJoinExample").getOrCreate()

# Read the Parquet files to get their schemas
schema1 = spark.read.parquet("hz_parties.parquet").schema
schema2 = spark.read.parquet("hz_party_sites.parquet").schema

# Identify timestamp columns in each schema
timestamp_columns1 = [field.name for field in schema1.fields if isinstance(field.dataType, TimestampType)]
timestamp_columns2 = [field.name for field in schema2.fields if isinstance(field.dataType, TimestampType)]

# Update the schemas to use a common timestamp type (e.g., TIMESTAMP(MICROS))
updated_schema1 = StructType([StructField(field.name, TimestampType(), field.nullable) if field.name in timestamp_columns1 else field for field in schema1.fields])
updated_schema2 = StructType([StructField(field.name, TimestampType(), field.nullable) if field.name in timestamp_columns2 else field for field in schema2.fields])

# Read Parquet files with updated schemas
df1 = spark.read.schema(updated_schema1).parquet("hz_parties.parquet")
df2 = spark.read.schema(updated_schema2).parquet("hz_party_sites.parquet")

# Perform the join
result_df = df1.join(df2, "PARTY_ID")

# Show the result or perform further operations
result_df.show()

# Stop the Spark session
spark.stop()


Py4JJavaError: An error occurred while calling o32.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (DESKTOP-63H1IAF executor driver): org.apache.spark.sql.AnalysisException: Illegal Parquet type: INT64 (TIMESTAMP(NANOS,false)).
	at org.apache.spark.sql.errors.QueryCompilationErrors$.illegalParquetTypeError(QueryCompilationErrors.scala:1826)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.illegalType$1(ParquetSchemaConverter.scala:206)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.$anonfun$convertPrimitiveField$2(ParquetSchemaConverter.scala:283)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.convertPrimitiveField(ParquetSchemaConverter.scala:224)
	at org.apache.spark.sql.execution.datasources.parquet.Par

In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.types import TimestampType

# Create a Spark session
spark = SparkSession.builder.appName("ParquetJoinExample").getOrCreate()

# Read the Parquet files to get their schemas
schema1 = spark.read.parquet("hz_parties.parquet").schema
schema2 = spark.read.parquet("hz_party_sites.parquet").schema

# Identify timestamp columns in each schema
timestamp_columns1 = [field.name for field in schema1.fields if isinstance(field.dataType, TimestampType)]
timestamp_columns2 = [field.name for field in schema2.fields if isinstance(field.dataType, TimestampType)]

print("Timestamp Columns in DataFrame 1:", timestamp_columns1)
print("Timestamp Columns in DataFrame 2:", timestamp_columns2)


Py4JJavaError: An error occurred while calling o39.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (DESKTOP-63H1IAF executor driver): org.apache.spark.sql.AnalysisException: Illegal Parquet type: INT64 (TIMESTAMP(NANOS,false)).
	at org.apache.spark.sql.errors.QueryCompilationErrors$.illegalParquetTypeError(QueryCompilationErrors.scala:1826)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.illegalType$1(ParquetSchemaConverter.scala:206)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.$anonfun$convertPrimitiveField$2(ParquetSchemaConverter.scala:283)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.convertPrimitiveField(ParquetSchemaConverter.scala:224)
	at org.apache.spark.sql.execution.datasources.parquet.Par